In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Uterine_Carcinosarcoma"
cohort = "GSE36138"

# Input paths
in_trait_dir = "../../input/GEO/Uterine_Carcinosarcoma"
in_cohort_dir = "../../input/GEO/Uterine_Carcinosarcoma/GSE36138"

# Output paths
out_data_file = "../../output/preprocess/Uterine_Carcinosarcoma/GSE36138.csv"
out_gene_data_file = "../../output/preprocess/Uterine_Carcinosarcoma/gene_data/GSE36138.csv"
out_clinical_data_file = "../../output/preprocess/Uterine_Carcinosarcoma/clinical_data/GSE36138.csv"
json_path = "../../output/preprocess/Uterine_Carcinosarcoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE36138_family.soft.gz', 'GSE36138_series_matrix.txt.gz']
Identified SOFT files: ['GSE36138_family.soft.gz']
Identified matrix files: ['GSE36138_series_matrix.txt.gz']



Background Information:
!Series_title	"SNP array data from the Cancer Cell Line Encyclopedia (CCLE)"
!Series_summary	"The Cancer Cell Line Encyclopedia (CCLE) project is a collaboration between the Broad Institute, the Novartis Institutes for Biomedical Research and the Genomics Novartis Foundation to conduct a detailed genetic and pharmacologic characterization of a large panel of human cancer models"
!Series_summary	"It consists of a compilation of gene expression, chromosomal copy number, and massively parallel sequencing data from nearly 1,000 human cancer cell lines. All raw and processed data are available through an integrated portal on www.broadinstitute.org/ccle"
!Series_overall_design	"The final cell line collection spans 36 cancer types. Representation of cell lines for each cancer type was mainly driven by cancer mortality in the United States, as a surrogate of unmet medical need, as well as availability."

Sample Characteristics Dictionary:
{0: ['primary site: central_ne

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze dataset for gene expression data
# Looking at the background information, this appears to be SNP array data from cancer cell lines
# rather than gene expression data suitable for our analysis
is_gene_available = False

# For trait data analysis: looking for Uterine_Carcinosarcoma in the dataset
# After reviewing the sample characteristics dictionary, we need to check if the data includes our trait of interest
trait_row = None

# From the sample characteristics, we can see "primary site: endometrium" is present
# But there's no specific mention of carcinosarcoma in the provided characteristics
# Looking through histology and histology subtype fields reveals no clear indicator of carcinosarcoma

# Age information is not provided in the sample characteristics
age_row = None

# Gender information is not provided in the sample characteristics
gender_row = None

# Define conversion functions (though they won't be used for this dataset)
def convert_trait(value):
    """Convert trait value to binary format."""
    if not value or pd.isna(value):
        return None
    value = value.split(':')[-1].strip().lower()
    if 'carcinosarcoma' in value:
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age value to continuous format."""
    if not value or pd.isna(value):
        return None
    value = value.split(':')[-1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender value to binary format."""
    if not value or pd.isna(value):
        return None
    value = value.split(':')[-1].strip().lower()
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# Check if trait data is available
is_trait_available = trait_row is not None

# Save metadata for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Skip clinical feature extraction since trait_row is None


False

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['14q(0)', '14q(I-1)', '14q(I-2)', '14q(I-3)', '14q(I-4)', '14q(I-5)',
       '14q(I-6)', '14q(I-7)', '14q(I-8)', '14q(I-9)', '14q(II-1)',
       '14q(II-10)', '14q(II-11)', '14q(II-12)', '14q(II-13)', '14q(II-14)',
       '14q(II-15)', '14q(II-16)', '14q(II-17)', '14q(II-18)'],
      dtype='object', name='ID')

Gene expression data shape: (22519, 947)


### Step 4: Gene Identifier Review

In [5]:
# Upon examining the gene identifiers, these are not standard human gene symbols
# These appear to be probe identifiers, possibly specific to a particular microarray platform
# Identifiers like '14q(0)', '14q(I-1)' are not standard gene symbols, which would typically be names like 'BRCA1', 'TP53', etc.
# These would need to be mapped to standard gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file that will contain the probe-to-gene mapping
try:
    # First, let's examine the SOFT file content to better understand its structure
    import gzip
    
    # Read the first 100 lines of the file to analyze its structure
    probe_id_pattern = None
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        sample_content = [f.readline().strip() for _ in range(100)]
    print("Sample content from SOFT file:")
    for line in sample_content[:10]:  # Print first 10 lines
        print(line)
    
    # Look for the platform or series information sections that might contain our probe mappings
    platform_lines = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if "^PLATFORM" in line or "^SERIES" in line or "!Platform_" in line:
                platform_lines.append(line.strip())
    
    print("\nPlatform/Series information:")
    for line in platform_lines[:10]:  # Print first 10 platform-related lines
        print(line)
    
    # Try using a different approach to get annotation data
    # Let's read the file line by line to find sections with probe mappings
    annotation_sections = []
    current_section = None
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith('^PLATFORM') or line.startswith('^SAMPLE'):
                if current_section:
                    annotation_sections.append(current_section)
                current_section = [line.strip()]
            elif current_section is not None:
                current_section.append(line.strip())
    
    if current_section:
        annotation_sections.append(current_section)
    
    # Print the beginning of each section to identify which might contain probe mappings
    print(f"\nFound {len(annotation_sections)} annotation sections")
    if annotation_sections:
        for i, section in enumerate(annotation_sections[:3]):  # Print first 3 sections
            print(f"\nSection {i} preview:")
            for line in section[:5]:  # Print first 5 lines of each section
                print(line)
    
    # For now, create an empty annotation dataframe
    # Once we identify the correct section, we can correctly parse the mapping
    gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

print("\nNote: Based on the file preview, this appears to be an SNP array dataset (not gene expression)")
print("The probe IDs seem to represent SNP locations not gene probes, which aligns with the dataset's")
print("description as 'SNP array data from the Cancer Cell Line Encyclopedia (CCLE)'")

Sample content from SOFT file:
^DATABASE = GeoMiame
!Database_name = Gene Expression Omnibus (GEO)
!Database_institute = NCBI NLM NIH
!Database_web_link = http://www.ncbi.nlm.nih.gov/geo
!Database_email = geo@ncbi.nlm.nih.gov
^SERIES = GSE36138
!Series_title = SNP array data from the Cancer Cell Line Encyclopedia (CCLE)
!Series_geo_accession = GSE36138
!Series_status = Public on Mar 20 2012
!Series_submission_date = Feb 28 2012



Platform/Series information:
^SERIES = GSE36138
^PLATFORM = GPL15315
!Platform_title = Affymetrix Genome-Wide Human SNP 6.0 Array
!Platform_geo_accession = GPL15315
!Platform_status = Public on Mar 20 2012
!Platform_submission_date = Mar 06 2012
!Platform_last_update_date = Jan 17 2013
!Platform_technology = in situ oligonucleotide
!Platform_distribution = commercial
!Platform_organism = Homo sapiens



Found 948 annotation sections

Section 0 preview:
^PLATFORM = GPL15315
!Platform_title = Affymetrix Genome-Wide Human SNP 6.0 Array
!Platform_geo_accession = GPL15315
!Platform_status = Public on Mar 20 2012
!Platform_submission_date = Mar 06 2012

Section 1 preview:
^SAMPLE = GSM887898
!Sample_title = 1321N1
!Sample_geo_accession = GSM887898
!Sample_status = Public on Mar 20 2012
!Sample_submission_date = Mar 06 2012

Section 2 preview:
^SAMPLE = GSM887899
!Sample_title = 143B
!Sample_geo_accession = GSM887899
!Sample_status = Public on Mar 20 2012
!Sample_submission_date = Mar 06 2012

Note: Based on the file preview, this appears to be an SNP array dataset (not gene expression)
The probe IDs seem to represent SNP locations not gene probes, which aligns with the dataset's
description as 'SNP array data from the Cancer Cell Line Encyclopedia (CCLE)'
